In [1]:
from codelin.models.const_tree import C_Tree
from codelin.encs.constituent import *
from codelin.utils.constants import *
from codelin.models.linearized_tree import LinearizedTree
from codelin.models.const_label import C_Label
from nltk.tree import Tree
from nltk.treetransforms import collapse_unary

import pandas as pd
import os
import re
import copy

def pt(t, d=True):
    '''
    Print tree if d is True
    '''
    if d:
        Tree.fromstring(str(t)).pretty_print()

def pb(ts, dir="r"):
    '''
    Print binary tree
    '''
    t = C_Tree.from_string(str(ts))
    t = t.collapse_unary()
    
    if dir == "r":
        t = C_Tree.to_binary_right(t)
    elif dir == "l":
        t = C_Tree.to_binary_left(t)
    else:
        raise ValueError("dir must be 'r' or 'l'")
    
    Tree.fromstring(str(t)).pretty_print()

def clean_features(t):
    '''
    remove all characters between ## in all the nodes of the tree
    '''
    regex = re.compile(r'##.*?##')
    label = regex.sub('', t.label)
    children = [clean_features(c) for c in t.children]
    return C_Tree(label, children)

##
# store trees that gave problems 
##

# genia
g01 = "(ROOT (NONE The) (NONE 4-carboranyl-substituted) (NONE compounds) (NONE -LRB-) (NONE 7) (NONE , ) (NONE 11) (NONE -RRB-) (NONE showed) (NONE antagonistic) (NONE activity) (NONE but) (NONE no) (NONE agonistic) (NONE activity) (NONE even) (NONE in) (NONE the) (NONE presence) (NONE of) (NONE the) (NONE potent) (NONE synergist) (NONE HX630) (NONE .))"
g02 = "(ROOT (NONE In) (NONE parallel) (NONE ,) (NONE NAC) (NONE was) (NONE shown) (NONE to) (NONE down-regulate) (NONE the) (NONE production) (NONE of) (protein (NONE cytokines)) (NONE by) (cell_type (NONE DC)) (NONE as) (NONE well) (NONE as) (NONE their) (NONE surface) (NONE expression) (NONE of) (protein (NONE HLA-DR)) (NONE ,) (protein (NONE CD86)) (NONE -LRB-) (protein (NONE B7-2)) (NONE -RRB-) (NONE ,) (NONE and) (protein (NONE CD40) (NONE molecules)) (NONE both) (NONE at) (NONE the) (NONE basal) (NONE state) (NONE and) (NONE upon) (NONE LPS) (NONE activation) (NONE .))"
g03 = "(ROOT (NONE The) (RNA (protein (NONE MNDA)) (NONE mRNA)) (NONE level) (NONE in) (cell_type (NONE primary) (NONE granulocytes)) (NONE was) (NONE unaffected) (NONE by) (NONE addition) (NONE of) (protein (NONE interferon) (NONE alpha)) (NONE and) (NONE other) (NONE agents) (NONE including) (protein (NONE interferon) (NONE gamma)) (NONE ,) (protein (NONE endotoxin)) (NONE ,) (protein (NONE poly-LRB-I-RRB-) (NONE poly-LRB-C-RRB-)) (NONE ,) (NONE and) (protein (NONE FMLP)) (NONE .))"
g04 = "(ROOT (NONE The) (NONE expression) (NONE of) (NONE c-fos) (NONE ,) (NONE c-jun) (NONE and) (NONE jun) (NONE B) (NONE proto-oncogenes) (NONE was) (NONE studied) (NONE in) (cell_line (protein (NONE phytohemagglutinin)) (NONE -LRB-) (protein (NONE PHA)) (NONE -RRB-) (NONE activated) (NONE peripheral) (NONE blood) (NONE lymphocytes)) (NONE -LRB-) (cell_type (NONE PBL)) (NONE -RRB-) (NONE from) (NONE young) (NONE and) (NONE aged) (NONE humans) (NONE .))"
g05 = "(ROOT (cell_line (NONE E2) (NONE -stimulated) (NONE ER+) (NONE cells)) (NONE were) (NONE more) (NONE susceptible) (NONE to) (NONE lysis) (NONE by) (cell_type (NONE LAK) (NONE cells)) (NONE than) (NONE corresponding) (cell_line (NONE TAM) (NONE -treated)) (NONE or) (cell_type (NONE control) (NONE cells)) (NONE ,) (NONE while) (NONE treatment) (NONE of) (cell_line (NONE ER-cells)) (NONE with) (NONE either) (NONE E2) (NONE or) (NONE TAM) (NONE alone) (NONE did) (NONE not) (NONE alter) (NONE from) (NONE control) (NONE their) (NONE susceptibility) (NONE to) (NONE this) (NONE immune-mediated) (NONE lysis) (NONE .))"
# ptb
p01 = "(S (S (NP (JJ Big) (NN investment) (NNS banks)) (VP (VBD refused) (S (VP (TO to) (VP (VB step) (ADVP (RB up) (PP (TO to) (NP (DT the) (NN plate)))) (S (VP (TO to) (VP (VB support) (NP (DT the) (JJ beleaguered) (NN floor) (NNS traders)) (PP (IN by) (S (VP (VBG buying) (NP (NP (JJ big) (NNS blocks)) (PP (IN of) (NP (NN stock))))))))))))))) (, ,) (NP (NNS traders)) (VP (VBP say)) (. .))"
p02 = "(SINV (S (ADVP (RB Once) (RB again)) (-LRB- -LCB-) (NP (DT the) (NNS specialists)) (-RRB- -RCB-) (VP (VBD were) (RB not) (ADJP (JJ able) (S (VP (TO to) (VP (VB handle) (NP (NP (DT the) (NNS imbalances)) (PP (IN on) (NP (NP (DT the) (NN floor)) (PP (IN of) (NP (DT the) (NNP New) (NNP York) (NNP Stock) (NNP Exchange)))))))))))) (, ,) ('' '') (VP (VBD said)) (NP (NP (NNP Christopher) (NNP Pedersen)) (, ,) (NP (NP (JJ senior) (NN vice) (NN president)) (PP (IN at) (NP (NNP Twenty-First) (NNP Securities) (NNP Corp))))) (. .))"
p03 = "(S (NP (NP (NNP Seven) (NNP Big) (NNP Board) (NNS stocks)) (: --) (NP (NP (NNP UAL)) (, ,) (NP (NNP AMR)) (, ,) (NP (NNP BankAmerica)) (, ,) (NP (NNP Walt) (NNP Disney)) (, ,) (NP (NNP Capital) (NNP Cities\/ABC)) (, ,) (NP (NNP Philip) (NNP Morris)) (CC and) (NP (NNP Pacific) (NNP Telesis) (NNP Group))) (: --)) (VP (VP (VBD stopped) (S (VP (NN trading)))) (CC and) (VP (ADVP (RB never)) (VBD resumed))) (. .))"
p04 = "(S (S (NP (JJ Big) (NN investment) (NNS banks)) (VP (VBD refused) (S (VP (TO to) (VP (VB step) (ADVP (RB up) (PP (TO to) (NP (DT the) (NN plate)))) (S (VP (TO to) (VP (VB support) (NP (DT the) (JJ beleaguered) (NN floor) (NNS traders)) (PP (IN by) (S (VP (VBG buying) (NP (NP (JJ big) (NNS blocks)) (PP (IN of) (NP (NN stock))))))))))))))) (, ,) (NP (NNS traders)) (VP (VBP say)) (. .))"
p05 = "(FRAG (SBAR (IN As) (IN in) (: :) (`` ``) (SQ (NP (PRP You)) (VP (VBD went) (NP (VBG ballooning))) (. ?) (. ?) (. !) (. !))))"
p06 = "(NP (NP (NN Year)) (VP (VBN ended) (NP (NNP Dec.) (CD 31) (, ,) (CD 1988))) (X (NN !)))"
p07 = "(S (INTJ (RB No)) (, ,) (NP (PRP it)) (VP (VBD was) (RB n't) (NP (NNP Black) (NNP Monday))) (. .))"
p08 = "(S (NP (DT The) (JJ finger-pointing)) (VP (VBZ has) (ADVP (RB already)) (VP (VBN begun))) (. .))"
p09 = "(S (`` ``) (NP (DT The) (NN equity) (NN market)) (VP (VBD was) (ADJP (JJ illiquid))) (. .))"
p10 = "(S (PP (IN In) (NP (NP (DT an) (NNP Oct.) (CD 19) (NN review)) (PP (IN of) (NP (`` ``) (NP (DT The) (NN Misanthrope)) ('' '') (PP (IN at) (NP (NP (NNP Chicago) (POS 's)) (NNP Goodman) (NNP Theatre))))) (PRN (-LRB- -LRB-) (`` ``) (S (NP (JJ Revitalized) (NNS Classics)) (VP (VBP Take) (NP (DT the) (NNP Stage)) (PP (IN in) (NP (NNP Windy) (NNP City))))) (, ,) ('' '') (NP (NNP Leisure) (CC &) (NNP Arts)) (-RRB- -RRB-)))) (, ,) (NP (NP (NP (DT the) (NN role)) (PP (IN of) (NP (NNP Celimene)))) (, ,) (VP (VBN played) (PP (IN by) (NP (NNP Kim) (NNP Cattrall)))) (, ,)) (VP (VBD was) (VP (ADVP (RB mistakenly)) (VBN attributed) (PP (TO to) (NP (NNP Christina) (NNP Haag))))) (. .))"
p11 = "(NP (NNP Conrail))"
p12 = "(S (NP (NP (RB Not) (PDT all) (DT those)) (SBAR (WHNP (WP who)) (S (VP (VBD wrote))))) (VP (VBP oppose) (NP (DT the) (NNS changes))) (. .))"
# spmrl
s01 = "(VROOT (PN (NE-PNC##lem=Grand|case=*|number=*|gender=*## GRAND) (NE-PNC##lem=Rapids|case=nom|number=sg|gender=neut## RAPIDS)) ($,##lem=--|_## ,) (NP (ADJA-NK##lem=3.|case=nom|number=sg|gender=masc|degree=pos## 3.) (NN-NK##lem=Januar|case=nom|number=sg|gender=masc## Januar)) ($LRB##lem=--|_## -LRB-) (NE##lem=afp|case=nom|number=sg|gender=*## afp) ($LRB##lem=--|_## -RRB-) ($.##lem=--|_## .))"
s02 = "(VROOT (S (PP-MO (APPR-AC##lem=in|_## In) (ART-NK##lem=der|case=dat|number=pl|gender=*## den) (NE-NK##lem=USA|case=dat|number=pl|gender=*## USA)) (VAFIN-HD##lem=haben|number=sg|person=3|tense=pres|mood=ind## hat) (PRF-OA##lem=sich|case=acc|number=sg|person=3## sich) (NP-SB (ART-NK##lem=ein|case=nom|number=sg|gender=neut## ein) (AP-NK (NP-AMS (CARD-NK##lem=neun|_## neun) (NN-NK##lem=Jahr|case=acc|number=pl|gender=neut## Jahre)) (ADJA-HD##lem=alt|case=nom|number=sg|gender=neut|degree=pos## altes)) (NN-NK##lem=Mädchen|case=nom|number=sg|gender=neut## Mädchen)) (CVP-OC (VP-CJ (ADJD-MO##lem=erfolgreich|degree=pos## erfolgreich) (PP-MO (APPR-AC##lem=als|_## als) (NN-NK##lem=Geburtshelferin|case=nom|number=sg|gender=fem## Geburtshelferin)) (VVPP-HD##lem=betätigen|_## betätigt)) (KON-CD##lem=und|_## und) (VP-CJ (VVPP-HD##lem=helfen|_## geholfen)))) ($,##lem=--|_## ,) (VP-OC (NP-OA (PPOSAT-NK##lem=sein|case=acc|number=sg|gender=masc## seinen) (ADJA-NK##lem=klein|case=acc|number=sg|gender=masc|degree=pos## kleinen) (NN-NK##lem=Bruder|case=acc|number=sg|gender=masc## Bruder)) (PP-MO (APPRART-AC##lem=zu|case=dat|number=sg|gender=fem## zur) (NN-NK##lem=Welt|case=dat|number=sg|gender=fem## Welt)) (VZ-HD (PTKZU-PM##lem=zu|_## zu) (VVINF-HD##lem=bringen|_## bringen))) ($.##lem=--|_## .))"
s03 = "(NE##lem=USA|case=nom|number=pl|gender=*## USA)"
s04 = "(VROOT (S (NP-SB (ART-NK##lem=der|case=nom|number=sg|gender=neut## Das) (NN-NK##lem=Ministerium|case=nom|number=sg|gender=neut## Ministerium)) (VAFIN-HD##lem=haben|number=sg|person=3|tense=pres|mood=ind## hat) (NP-OA (AVP-MO (ADV-HD##lem=nur|_## nur) (ADV-MO##lem=noch|_## noch)) (ART-NK##lem=der|case=acc|number=sg|gender=fem## die) (NN-NK##lem=Fachaufsicht|case=acc|number=sg|gender=fem## Fachaufsicht))) ($.##lem=--|_## .))"

/home/droca1/.conda/envs/tf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
encoder = C_JuxtaposedEncoding(separator="[_]", unary_joiner="[+]", reverse=False, binary=False, binary_direction="L", binary_marker="[b]")
treebank_path = "/home/droca1/Treebanks/const/PENN_TREEBANK/train.trees"
sample_trees = open(treebank_path).readlines()
for idx, ts in enumerate(sample_trees):
    ts = ts.rstrip()
    t = C_Tree.from_string(ts)
    lc = encoder.encode(t)
    dt = encoder.decode(lc)
    assert str(ts)==str(dt), f"Error at {idx}\n{lc}\n{ts}\n{dt}"

In [11]:
gaps_encoder = C_GapsEncoding(separator="[_]", unary_joiner="[+]", reverse=False, binary=True, binary_direction="R", binary_marker="[b]")

treebank_path = "/home/droca1/Treebanks/const/PENN_TREEBANK/train.trees"
sample_trees = open(treebank_path).readlines()
label_set = set()
for idx, ts in enumerate(sample_trees):
    ts = ts.rstrip()
    ts = C_Tree.from_string(ts)
    lc = gaps_encoder.encode(ts)
    dt = gaps_encoder.decode(lc)
    assert str(ts)==str(dt), f"Error at {idx}\n{lc}\n{ts}\n{dt}"

In [15]:
tetra_encoder = C_Tetratag(separator="[_]", unary_joiner="[+]", reverse=False, binary_direction="R", binary_marker="[b]")
treebank_path = "/home/droca1/Treebanks/const/PENN_TREEBANK/train.trees"
sample_trees = open(treebank_path).readlines()[:10]
label_set = set()
for idx, ts in enumerate(sample_trees):
    ts = ts.rstrip()
    ts = C_Tree.from_string(ts)
    lc = tetra_encoder.encode(ts, mode='preorder')
    dt = tetra_encoder.decode(lc)
    assert str(ts)==str(dt), f"Error at {idx}\n{lc}\n{ts}\n{dt}"


IndexError: list index out of range

### NER2Trees


In [ ]:
from ner2trees import *

g01j = '{"tokens": ["The","binding","of","interleukin-2","(","IL-2",")","to","its","receptor","on","normal","T","cells","induces","nuclear","expression","of","nuclear","factor","kappaB","(","NF-kappaB",")",",","activation","of","the","IL-2","receptor","(","IL-2R",")","alpha","chain","gene",",","and","cell","proliferation","."],"doc_id": "MEDLINE:98281217","sent_id": "MEDLINE:98281217-1","entity_mentions": [{"start": 3,"end": 4,"entity_type": "protein","text": "interleukin-2"},{"start": 5,"end": 6,"entity_type": "protein","text": "IL-2"},{"start": 12,"end": 14,"entity_type": "cell_type","text": "T cells"},{"start": 18,"end": 21,"entity_type": "protein","text": "nuclear factor kappaB"},{"start": 22,"end": 23,"entity_type": "protein","text": "NF-kappaB"},{"start": 28,"end": 29,"entity_type": "protein","text": "IL-2"},{"start": 28,"end": 30,"entity_type": "protein","text": "IL-2 receptor"},{"start": 28,"end": 35,"entity_type": "protein","text": "IL-2 receptor ( IL-2R ) alpha chain"},{"start": 28,"end": 36,"entity_type": "DNA","text": "IL-2 receptor ( IL-2R ) alpha chain gene"}]}'

line = g01j
s = json.loads(line)
words, postags, entities = s['tokens'], ["NONE"]*len(s['tokens']), parse_entities(s['entity_mentions'], len(s['tokens']),  extractor=parse_genia) 
t = entities_to_tree(words, postags, entities)
t.clean_tree()
Tree.fromstring(str(t)).pretty_print()

                                                                                                                                                         ROOT                                                                                                                                              
  ________________________________________________________________________________________________________________________________________________________|_____________________________________________________________________________________________________________________________________________    
 |      |     |         |         |      |      |    |    |      |      |     |             |              |       |        |       |              |             |       |       |    |       |       |    |                                        DNA                    |    |    |         |        |  
 |      |     |         |         |      |      |    |    |      |      |     |             |      

### Tetratag-Several-Orders

In [ ]:
## Transition based operators

def shift_action(node):
    '''
    For SHIFT actions, we encode whether the
    node being shifted is a left or a right child of
    its parent.
    '''
    label = "r" if node.is_left_child() else "l"
    return label

def reduce_action(node):
    '''
    For REDUCE actions, we encode the identity
    of the non-terminal being reduced as well as
    whether it is a left or a right child
    '''
    label =  "R" if node.is_left_child() else "L"
    label += (">"+node.label)
    return label

def combine(tree, new_child):
    '''
    Replaces a C_NONE_LABEL inside 'tree'
    with new_child
    '''
    # trees should have only 2 child nodes
    if type(new_child) is str:
        new_child = C_Tree(new_child)
    
    current_level = tree
    
    while(not current_level.has_none_child()):
        if (len(current_level.children) < 1):
            print("[MERGE ERROR] No children found")
            return tree
        current_level = current_level.r_child()
    
    if current_level.children[0].label == C_NONE_LABEL:
        current_level.children[0] = new_child
    elif current_level.children[1].label == C_NONE_LABEL:
        current_level.children[1] = new_child
    return tree


ts = p06
ts = p04
ts = p05
## Encoding in-order
print("INORDER:")
tree = C_Tree.from_string(ts)
tree = tree.collapse_unary()
tree = C_Tree.to_binary_right(tree)
pt(tree)

tags    = [n.label for n in tree.get_preterminals()]
words   = [n.label for n in tree.get_terminals()]

tree = tree.remove_preterminals()
nodes = []
C_Tree.inorder(tree, lambda x: nodes.append(x))
l_in = []
cl = []
for node in nodes:
    if node.is_terminal():
        cl.append(shift_action(node))
    else:
        cl.append(reduce_action(node))
        
        l_in.append("_".join(cl))
        cl = []

# last reduce action
l_in.append("_".join(cl))
cl = []


## Encoding pre-order
print("PREORDER:")
tree = C_Tree.from_string(ts)
tree = tree.collapse_unary()
tree = C_Tree.to_binary_right(tree)
pt(tree)

tags    = [n.label for n in tree.get_preterminals()]
words   = [n.label for n in tree.get_terminals()]

tree = tree.remove_preterminals()
nodes = []
C_Tree.preorder(tree, lambda x: nodes.append(x))
l_pr = []
cl = []
for node in nodes:    
    if node.is_terminal():
        cl.append(shift_action(node))   
        l_pr.append("_".join(cl))
        cl = []
    else:
        cl.append(reduce_action(node))


## Encoding post-order
print("POSTORDER:")
tree = C_Tree.from_string(ts)
tree = tree.collapse_unary()
tree = C_Tree.to_binary_left(tree)
pt(tree)

tags    = [n.label for n in tree.get_preterminals()]
words   = [n.label for n in tree.get_terminals()]

tree = tree.remove_preterminals()
nodes = []
C_Tree.postorder(tree, lambda x: nodes.append(x))
l_po = []
cl = []
for node in nodes:
    if node.is_terminal():
        cl.append(shift_action(node))
        l_po.append("_".join(cl))
        cl = []
    else:
        cl.append(reduce_action(node))
# last non-terminal encoding
l_po[-1]+="_"+"_".join(cl)


print(f"{str('word'):<20} {str('postag'):<20} {str('inorder'):<20} {str('preorder'):<20} {str('postorder'):<20}")
print("-"*100)
for i in range(len(l_in)):
    print(f"{str(words[i]):<20} {str(tags[i]):<20} {str(l_in[i]):<20} {str(l_pr[i]):<20} {str(l_po[i]):<20}")


## Decoding for in-order
stack = []
buffer = copy.deepcopy(words)
for t, l in zip(tags, l_in):
    operators = l.split("_")
    for operator in operators:
        if operator == 'r':
            w = buffer.pop(0)
            terminal_tree = C_Tree(t, children=[C_Tree(w)])
            stack.append(terminal_tree)
        
        elif operator == 'l':
            w = buffer.pop(0)
            terminal_tree = C_Tree(t, children=[C_Tree(w)])
            
            if len(stack)==0:
                stack.append(terminal_tree)
            else:                
                current_level = tree                
                stack[-1] = combine(stack[-1], terminal_tree)

        elif operator.startswith("R"):
            nt = operator.split(">")[1]
            tree = C_Tree(nt, [stack[-1], C_Tree.empty_tree()])
            
            if len(stack)==0:
                stack.append(tree)
            else:
                stack[-1] = tree

        elif operator.startswith("L"):
            nt = operator.split(">")[1]
            tree = stack.pop()
            tree = C_Tree(nt, [tree, C_Tree.empty_tree()])
            
            if len(stack)==0:
                stack.append(tree)
            else:
                stack[-1] = combine(stack[-1], tree)

final_tree = stack.pop()
print("INORDER DECODE:")
pt(final_tree)


## Decoding for pre-order
stack = []
buffer = copy.deepcopy(words)
for t, l in zip(tags, l_pr):
    operators = l.split("_")
    
    for operator in operators:
        if operator == 'r':
            # node is a left terminal children, add it to the stack
            w = buffer.pop(0)
            stack[-1].children[0] = C_Tree(t, children=[C_Tree(w)])
        
        elif operator == 'l':
            # node is a right terminal child, combine it with the top of the stack
            w = buffer.pop(0)
            terminal_tree = C_Tree(t, children=[C_Tree(w)])
            parent_tree = stack.pop()
            parent_tree.children[1] = terminal_tree

            # moving up the tree until we find a node with a right child
            while not parent_tree.has_none_child() and len(stack)>0:
                parent_tree = stack.pop()
                parent_tree.update_custody()
            
            stack.append(parent_tree)

        elif operator.startswith("R"):
            nt = operator.split(">")[1]
            
            # node is a left non terminal child, combine it with the top of the stack
            if len(stack)>0:
                l_child = C_Tree(nt, children=[C_Tree(C_NONE_LABEL), C_Tree(C_NONE_LABEL)])
                stack[-1].children[0] = l_child
                stack[-1].update_custody()
                
                stack.append(l_child)
            else:
                stack.append(C_Tree(nt, [C_Tree.empty_tree(), C_Tree.empty_tree()]))

        elif operator.startswith("L"):
            nt = operator.split(">")[1]
            
            # node is a right non terminal child, combine it with the top of the stack
            r_child = C_Tree(nt, [C_Tree.empty_tree(), C_Tree.empty_tree()])
            stack[-1].children[1] = r_child
            stack[-1].update_custody()
            
            stack.append(r_child)

final_tree = stack.pop()
print("PREORDER DECODE:")
pt(final_tree)


## Decoding for postorder-order
stack = []
buffer = copy.deepcopy(words)
for t, l in zip(tags, l_po):
    operators = l.split("_")
    
    for operator in operators:
        current_word = buffer[0] if len(buffer)>0 else None
        
        if operator == 'r':
            w = buffer.pop(0)
            p_tree = C_Tree(t, children=[C_Tree(w)])
            stack.append(p_tree)
        
        elif operator == 'l':
            # node is a right terminal child, combine it with the top of the stack
            w = buffer.pop(0)
            terminal_tree = C_Tree(t, children=[C_Tree(w)])
            
            # See if we can close this node
            if not stack[-1].has_none_child():
                stack.append(terminal_tree)    
                continue
            else:
                parent_tree = stack.pop()
                parent_tree.children[1] = terminal_tree

            # moving up the tree until we find a node with a right child
            while not parent_tree.has_none_child() and len(stack)>0:
                parent_tree = stack.pop()
                parent_tree.update_custody()
            
            stack.append(parent_tree)

        elif operator.startswith("R"):
            nt = operator.split(">")[1]
            
            # node is a left non terminal child, combine it with the top of the stack
            if len(stack)>0:
                r_child = stack.pop()
                l_child = stack.pop()
                p_tree  = C_Tree(nt, children=[l_child, r_child])
                p_tree.update_custody()
                
                stack.append(p_tree)
            else:
                stack.append(C_Tree(nt, [C_Tree.empty_tree(), C_Tree.empty_tree()]))

        elif operator.startswith("L"):
            nt = operator.split(">")[1]
            
            # node is a right non terminal child, combine it with the top of the stack
            if len(stack)>0:
                r_child = stack.pop()
                l_child = stack.pop()
                p_tree  = C_Tree(nt, children=[l_child, r_child])
                p_tree.update_custody()

                stack.append(p_tree)
            

final_tree = stack.pop()
print("POSTORDER DECODE:")
pt(final_tree)


INORDER:
    FRAG+SBAR                                                                             
  ______|_________                                                                         
 |            FRAG+SBAR*                                                                  
 |       _________|__________                                                              
 |      |                FRAG+SBAR*                                                       
 |      |          __________|__________                                                   
 |      |         |                 FRAG+SBAR*                                            
 |      |         |           __________|_______                                           
 |      |         |          |                  SQ                                        
 |      |         |          |           _______|___________________                       
 |      |         |          |          |                          SQ*      